# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from scipy.signal import butter, filtfilt
import random

In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[-2.10519694e-02, -2.64618150e-03, -1.09052584e-02, ...,
          -1.14497980e-02,  0.00000000e+00, -1.33858034e-02],
         [-1.99002311e-01, -2.20890388e-01, -2.76137829e-01, ...,
          -2.59782881e-01, -2.09464088e-01, -1.87755451e-01],
         [-2.39491284e-01, -2.51040578e-01, -2.89648235e-01, ...,
          -2.82749534e-01, -2.52160460e-01, -2.39700243e-01],
         ...,
         [-9.98796940e-01, -9.98772740e-01, -9.98712122e-01, ...,
          -9.96163070e-01, -9.97696698e-01, -9.98156428e-01],
         [-9.99429047e-01, -9.99434769e-01, -9.99466956e-01, ...,
          -9.99509513e-01, -9.99450743e-01, -9.99419093e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.80637967e-02, -3.36689858e-08, -7.33598135e-03, ...,
          -1.05369762e-02, -8.22041358e-04, -1.49706351e-02],
         [-1.93169340e-01, -2.11562306e-01, -2.58488506e-01, ...,
   

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128, 87)

array([[-1.73581708e-02,  0.00000000e+00, -8.14180914e-03, ...,
        -9.85445548e-03, -1.71658248e-04, -1.32468548e-02],
       [-1.93985939e-01, -2.12422282e-01, -2.69561797e-01, ...,
        -2.80415505e-01, -2.17691883e-01, -1.94388315e-01],
       [-2.68526703e-01, -2.86887318e-01, -3.42721850e-01, ...,
        -3.53653818e-01, -2.91956156e-01, -2.68852562e-01],
       ...,
       [-9.99117315e-01, -9.99106705e-01, -9.99057472e-01, ...,
        -9.98603582e-01, -9.98687506e-01, -9.98692393e-01],
       [-9.99352336e-01, -9.99339044e-01, -9.99306679e-01, ...,
        -9.99599993e-01, -9.99699354e-01, -9.99706149e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(12565, 20, 87)

...

(12565, 84, 87)

array([[-2.13950068e-01, -2.38894999e-01, -3.80664200e-01, ...,
        -1.41635209e-01, -1.01275146e-02, -1.78813934e-07],
       [-4.45840538e-01, -5.22175133e-01, -6.68615699e-01, ...,
        -4.22790468e-01, -2.85212219e-01, -2.24623069e-01],
       [-7.63046622e-01, -7.80026078e-01, -9.83664155e-01, ...,
        -7.36702800e-01, -5.25470257e-01, -5.25247097e-01],
       ...,
       [-2.00844555e+01, -7.19054260e+01, -6.65617599e+01, ...,
        -4.13212929e+01, -4.68384933e+01, -2.91117020e+01],
       [-2.03201313e+01, -6.45224457e+01, -6.51016922e+01, ...,
        -4.41159325e+01, -4.89345970e+01, -3.10661354e+01],
       [-2.05067978e+01, -6.17005310e+01, -5.89219780e+01, ...,
        -5.76679039e+01, -5.61718521e+01, -3.33402481e+01]])

'id'

(12565,)

'XC483906.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128, 87)

array([[-1.19495122e-02,  0.00000000e+00, -5.92639437e-03, ...,
        -9.61586274e-03, -1.45224054e-04, -1.35156866e-02],
       [-1.32104889e-01, -1.48279309e-01, -2.04118401e-01, ...,
        -2.71819055e-01, -2.12780029e-01, -1.90217406e-01],
       [-1.77966341e-01, -1.93037197e-01, -2.48268843e-01, ...,
        -3.41875643e-01, -2.82413810e-01, -2.60217577e-01],
       ...,
       [-8.56934249e-01, -8.21815670e-01, -8.13137054e-01, ...,
        -9.90914822e-01, -9.94373798e-01, -9.93971646e-01],
       [-9.33930933e-01, -9.18021381e-01, -9.22511458e-01, ...,
        -9.97301877e-01, -9.96644974e-01, -9.95461941e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -9.97628629e-01, -9.96140599e-01]])

'mfcc'

(3318, 20, 87)

...

(3318, 84, 87)

array([[ -0.81876791,  -0.83223093,  -0.95862055, ...,  -1.04782999,
         -0.91178417,  -0.89874911],
       [ -1.06328416,  -1.15007341,  -1.30937183, ...,  -1.50782466,
         -1.36429203,  -1.29628861],
       [ -1.31883109,  -1.31582081,  -1.49792707, ...,  -2.05177641,
         -1.80057609,  -1.77171004],
       ...,
       [-18.08393478, -52.00915909, -46.19884491, ..., -56.79980087,
        -66.42909241, -28.33953667],
       [-17.75449371, -48.76427078, -46.56741714, ..., -68.17790222,
        -62.38970947, -32.60255051],
       [-17.46328926, -54.79916   , -53.8106575 , ..., -59.53269958,
        -60.99742126, -34.56028748]])

'id'

(3318,)

'XC428404.mp3'

## **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    # val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = None # roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted') Look at documentation
    # print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(12565, 21228)

In [18]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 21228)

Fit the model with training data

In [19]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['all_features'] = a
val_results['all_features'] = b

In [ ]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

### **With MFCCs only**

In [ ]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['mfcc'] = a
val_results['mfcc'] = b

In [ ]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

### **With Mel-Spectrogram Only**

In [ ]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['melspectrogram'] = a
val_results['melspectrogram'] = b

In [ ]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With MFCCs and Mel-Spectrogram**

In [ ]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['mfcc_melspectrogram'] = a
val_results['mfcc_melspectrogram'] = b

In [ ]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With MFCCs and Chroma**

In [ ]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['mfcc_chroma'] = a
val_results['mfcc_chroma'] = b

In [ ]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With MFCCs and CQT**

In [ ]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['mfcc_cqt'] = a
val_results['mfcc_cqt'] = b

In [ ]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With Mel-Spectrogram and Chroma**

In [ ]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['melspectrogram_chroma'] = a
val_results['melspectrogram_chroma'] = b

In [ ]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With Mel-Spectrogram and CQT**

In [ ]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['melspectrogram_cqt'] = a
val_results['melspectrogram_cqt'] = b

In [ ]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With Mel-Spectrogram, MFCCs, CQT**

In [ ]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['melspectrogram_cqt_mfcc'] = a
val_results['melspectrogram_cqt_mfcc'] = b

In [ ]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

## **With Mel-Spectrogram, MFCCs, Chroma**

In [ ]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

In [ ]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

In [ ]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

In [ ]:
a = model.score(training_features, train_y)
b = model.score(validation_features, val_y)

print('Training accuracy:', a)
print('Validation accuracy:', b)

train_results['melspectrogram_chroma_mfcc'] = a
val_results['melspectrogram_chroma_mfcc'] = b

In [ ]:
val_scores['melspectrogram_chroma_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

# Review results from all models

In [ ]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

In [ ]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)